# Ejercicios 5-6

5. (Entrega obligatoria individual en repo) Genere un video en un patio o en un hall de edificio donde en un principio se vea vacío y luego aparezca una persona. Mediante los métodos de motion detection (sin usar deep learning) logre una detección de la persona cuando entra al cuadro suponiendo la utilidad para una cámara de seguridad. 
Luego sobre el mismo video aplique los algoritmos de flujo denso y disperso que se mostraron en clase. 
Escriba una reflexión sobre los resultados en el formato md dentro del Jupyter Notebook.
6. (Entrega obligatoria individual en repo) Explique cuál es diferencia entre localización de objetos y clasificación de imágenes. Muestre ejemplos de ello.

### Frame difference

In [1]:
import cv2 as cv
import numpy as np

--------------------
Sin morfologia

In [2]:
# Sin morfologia
def process_frame_difference(new_image, prev_image, **kwargs):
    
    new_gray = cv.cvtColor(new_image, cv.COLOR_RGB2GRAY) # Convertir las imágenes a escala de grises
    prev_gray = cv.cvtColor(prev_image, cv.COLOR_RGB2GRAY)

    frame_diff = cv.absdiff(new_gray, prev_gray) # Calcular la diferencia absoluta entre los fotogramas actual y anterior

    norm_diff = cv.normalize(frame_diff, None, 0, 255, cv.NORM_MINMAX) # Normalizar la imagen de diferencia

    _, thresh = cv.threshold(norm_diff, 150, 255, cv.THRESH_BINARY) # Umbralizar la imagen para resaltar las diferencias

    thresh_color = cv.cvtColor(thresh, cv.COLOR_GRAY2RGB)  # Convertir la imagen umbralizada a color para mantener la consistencia con el video original

    return thresh_color

In [3]:
video_path = "camara.mp4"
cap = cv.VideoCapture(video_path)

ret, prev_frame = cap.read()
prev_frame = cv.cvtColor(prev_frame, cv.COLOR_BGR2RGB)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    # Procesar el fotograma actual utilizando la función process_frame_difference
    processed_frame = process_frame_difference(frame_rgb, prev_frame)

    cv.imshow('Procesamiento de Frame Difference', processed_frame)
    cv.imshow('Video original', frame_rgb)

    prev_frame = frame_rgb

    if cv.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

--------------
Utilizadn odilatacion y contornos.

In [4]:
def draw_contours(image, contours):
    """Dibuja los contornos sobre la imagen."""
    result_image = image.copy()
    cv.drawContours(result_image, contours, -1, (0, 255, 0), 2)
    return result_image

In [5]:
def process_frame_difference_full(new_image, prev_image, **kwargs):
    new_gray = cv.cvtColor(new_image, cv.COLOR_RGB2GRAY) # Convertir las imágenes a escala de grises
    prev_gray = cv.cvtColor(prev_image, cv.COLOR_RGB2GRAY)

    frame_diff = cv.absdiff(new_gray, prev_gray) # Calcular la diferencia absoluta entre los fotogramas actual y anterior

    norm_diff = cv.normalize(frame_diff, None, 0, 255, cv.NORM_MINMAX)    # Normalizar la imagen de diferencia

    _, thresh = cv.threshold(norm_diff, 150, 255, cv.THRESH_BINARY)     # Umbralizar la imagen para resaltar las diferencias

    kernel = np.ones((5,5),np.uint8)  # Dilatar la imagen umbralizada para mejorar la detección de contornos
    dilated = cv.dilate(thresh, kernel, iterations = 1)

    dilated = dilated.astype(np.uint8) # Convertir la imagen dilatada a formato adecuado para findContours

    contours, _ = cv.findContours(dilated, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)     # Encontrar contornos en la imagen dilatada

 
    if kwargs.get('draw_mode', 0) == 0:    # Dibujar cuadros delimitadores alrededor de los contornos
      result_image = draw_contours(new_image, contours)
    elif kwargs.get('draw_mode', 0) == 1:
      result_image = draw_contours(thresh, contours)

    return result_image

In [7]:
video_path = "camara.mp4"
cap = cv.VideoCapture(video_path)

ret, prev_frame = cap.read()
prev_frame = cv.cvtColor(prev_frame, cv.COLOR_BGR2RGB)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    processed_frame = process_frame_difference_full(frame_rgb, prev_frame, draw_mode=0)  # Procesar el fotograma actual utilizando la función process_frame_difference_full

    cv.imshow('Procesamiento de Frame Difference', processed_frame)
    cv.imshow('Video original', frame_rgb)

    prev_frame = frame_rgb

    if cv.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

### Optical flow

-------------
Flujo optico disperso

In [8]:
def process_sparse_optical_flow(new_image, prev_image):
    
    new_gray = cv.cvtColor(new_image, cv.COLOR_BGR2GRAY) # Preparamos las imagenes de trabajo
    prev_gray_image = cv.cvtColor(prev_image, cv.COLOR_BGR2GRAY)

    # Verificar si ya se han detectado las características de Shi-Tomasi
    if not hasattr(process_sparse_optical_flow, "shi_tomasi_done"):
        # Definir parámetros para la detección de esquinas de Shi-Tomasi
        feature_params = dict(maxCorners=200, qualityLevel=0.05, minDistance=20, blockSize=7) # Puedo  ajustar estos valores para detectar solo la persona Reducir, Redcuir, aumentar.
        # Detectar puntos característicos en la imagen
        process_sparse_optical_flow.prev_points = cv.goodFeaturesToTrack(new_gray, mask=None, **feature_params)
        # Crear una máscara para dibujar el flujo óptico
        process_sparse_optical_flow.mask = np.zeros_like(new_image)
        # Marcar que se ha completado la detección de Shi-Tomasi
        process_sparse_optical_flow.shi_tomasi_done = True

    # Continuar si se ha completado la detección de Shi-Tomasi
    if process_sparse_optical_flow.shi_tomasi_done:
        prev_points = process_sparse_optical_flow.prev_points
        mask = process_sparse_optical_flow.mask

    # Parámetros para el flujo óptico de Lucas-Kanade
    lk_params = dict(winSize=(15, 15), maxLevel=2,
                     criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03)) # Achicar la ventana puede ayudar, mezclar el otro.

    # Calcular el flujo óptico de Lucas-Kanade
    new_points, status, error = cv.calcOpticalFlowPyrLK(prev_gray_image, new_gray, prev_points, None, **lk_params)
    # Filtrar puntos buenos
    good_old = prev_points[status == 1]
    good_new = new_points[status == 1]
    color = (100, 255, 0)  # Color para el dibujo
    # Dibujar el movimiento (flujo óptico)
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.astype(int).ravel()
        c, d = old.astype(int).ravel()
        mask = cv.line(mask, (a, b), (c, d), color, 2)
        new_image = cv.circle(new_image, (a, b), 3, color, -1)

    # Combinar la imagen actual con las líneas de flujo óptico dibujadas
    output = cv.add(new_image, mask)
    # Actualizar puntos para el siguiente cuadro
    process_sparse_optical_flow.prev_points = good_new.reshape(-1, 1, 2)
    return output

In [9]:
# Procesamiento de video
video_path = "camara.mp4"
cap = cv.VideoCapture(video_path)

ret, prev_frame = cap.read()
prev_frame = cv.cvtColor(prev_frame, cv.COLOR_BGR2RGB)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    # Procesar el fotograma actual utilizando la función process_sparse_optical_flow
    processed_frame = process_sparse_optical_flow(frame_rgb, prev_frame)

    cv.imshow('Procesamiento de Flujo Óptico', processed_frame)
    cv.imshow('Video original', frame_rgb)

    prev_frame = frame_rgb

    if cv.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

---------------
Flujo optico denso

In [109]:
# Función para procesar el flujo óptico denso
def process_dense_optical_flow(new_image, prev_image):
    # Convierte la nueva imagen a escala de grises
    gray = cv.cvtColor(new_image, cv.COLOR_BGR2GRAY)

    if not hasattr(process_dense_optical_flow, "init_done"):
        process_dense_optical_flow.prev_gray = cv.cvtColor(new_image, cv.COLOR_BGR2GRAY)
        process_dense_optical_flow.mask = np.zeros_like(new_image)
        process_dense_optical_flow.mask[..., 1] = 255
        process_dense_optical_flow.init_done = True

    if process_dense_optical_flow.init_done:
        prev_gray = process_dense_optical_flow.prev_gray
        mask = process_dense_optical_flow.mask

    # Calcula el flujo óptico
    flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0) # Se puede modificar los valores

    # Computa magnitud y ángulo de los vectores 2D
    magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])

    # Establece el tono de la imagen según la dirección del flujo óptico
    mask[..., 0] = angle * 180 / np.pi / 2

    # Establece el valor de la imagen según la magnitud del flujo óptico
    mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)

    # Convierte de HSV a RGB
    rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)
    
    # Actualiza la imagen previa a gris
    process_dense_optical_flow.prev_grayprev_gray = gray.copy()
    return rgb

In [110]:
# Procesamiento de video
video_path = "terraza.mp4"
cap = cv.VideoCapture(video_path)

ret, prev_frame = cap.read()
prev_frame = cv.cvtColor(prev_frame, cv.COLOR_BGR2RGB)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    # Procesar el fotograma actual utilizando la función process_sparse_optical_flow
    processed_frame = process_dense_optical_flow(frame_rgb, prev_frame)

    cv.imshow('Procesamiento de Flujo Denso', processed_frame)
    cv.imshow('Video original', frame_rgb)

    prev_frame = frame_rgb

    if cv.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

### Reflexion + Ejercicio 6

#### Reflexion de resultados.
_Los resultados son muy dependientes de que tan fija esté la cámara, de los movimientos que existan también. Dependen demasiado de los umbrales, y al ir ajustandolos recibimos distintos resultados. Sufrí mucho ajustando unos y otros, grabando diferentes videos y fue lo mejor que conseguí. Entiendo que por su naturaleza nunca va a ser perfecto salvo que el video lo sea._

#### Diferencia e/ Localizacion de objetos y Clasifiacion de Imagenes

_Por lo que pude entender de la teoría, la principal diferencia la vemos en el enfoque._
_La localización de objetos identifica y delimita un objeto en una imagen y su objetivo, entiendo, es identificar que ahí 'posicion' hay un objeto, es más importante saber dónde está el objeto en la imágen._
_Por ejemplo, detectar los autos en el video de ejemplo_

_Mientras que en la clasifiación de imágenes el enfoque cambia, refiere más a asignar una etiqueta a lo que se encuentra en la misma.El objetivo principal es más el de identificar la categoría o etiqueta a la que pertenece la imágen en conjunto. Más simple, entiendo que su principal tarea es asignar etiquetas a lo que encontramos, por ejempo, pajaro, perro, gato._

![alt text](cellphone.png)